
# Modelos discretos de Depredador-Presa


In [1]:
Pkg.add("LsqFit")

Pkg.update()



using PyPlot, LaTeXStrings, TaylorSeries, LsqFit

include("AutomDiff_V2.jl")
import AD: Dual, xdual

INFO: Nothing to be done
INFO: METADATA might be out-of-date — you may not have the latest version of LsqFit
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Updating METADATA...


LoadError: LoadError: METADATA cannot be updated. Resolve problems manually in /Users/Nina/.julia/v0.5/METADATA.
GitError(Code:ERROR, Class:Net, curl error: Couldn't resolve host 'github.com'
)
 in macro expansion at ./libgit2/error.jl:99 [inlined]
 in #fetch#52(::Base.LibGit2.FetchOptions, ::String, ::Function, ::Base.LibGit2.GitRemote, ::Array{AbstractString,1}) at ./libgit2/remote.jl:70
 in (::Base.LibGit2.#kw##fetch)(::Array{Any,1}, ::Base.LibGit2.#fetch, ::Base.LibGit2.GitRemote, ::Array{AbstractString,1}) at ./<missing>:0
 in #fetch#93(::String, ::String, ::Array{AbstractString,1}, ::Nullable{Base.LibGit2.AbstractCredentials}, ::Function, ::Base.LibGit2.GitRepo) at ./libgit2/libgit2.jl:164
 in (::Base.Pkg.Entry.##35#41)(::Base.LibGit2.GitRepo) at ./pkg/entry.jl:371
 in with(::Base.Pkg.Entry.##35#41, ::Base.LibGit2.GitRepo) at ./libgit2/types.jl:638
 in update(::String, ::Set{String}) at ./pkg/entry.jl:354
 in (::Base.Pkg.Dir.##2#3{Array{Any,1},Base.Pkg.Entry.#update,Tuple{String,

Los Modelos de Cazador-Presa son aquellos que representan la interacción de dos especies, una como cazador y otra como presa utilziados principalmente en ecología y biología pero con un sinúmero de aplicaciones en otras áreas de física, química y hasta economía.

En ecología se llaman modelos de "Depredador—presa" o de "planta—hervíboro". En este trabajo se les llama, modelos de "Depredador-Presa".

## Un modelo simple

Siguiendo la línea del libro de  [_Allman & Rhodes_](http://bookzz.org/book/436360/381bdc "Allman, Elizabeth S.; Rhodes, John A. 2003. Mathematical models in biology. An introduction") usaremos un modelo simple en este trabajo:

Se tienen dos especies, una de la cual es presa de la otra. También asumimos que la interacción entre las dos especies es el factor primario para determinar el tamaño de las poblaciones.

Denotamos $P_t$ el tamaño de la población de presas y $Q_t$ el tamaño de la población de de depredadores en un tiempo $t$, el cambio en la población de las dos especies está interrelacionado:

$$
\begin{aligned}
\Delta P &= F(P,Q)\\
\Delta Q &= G(P,Q)
\end{aligned}
$$

Ahora, para definir las funciones $F$ y $G$ de cambio pensemos en el caso en que no hubises predadores, la población de presas crecería hasta un punto en el que no hubiese más lugar o recursos, lo cual es modelado por la función logística:

$$
\Delta P = rP\left( 1-P\right)
$$

En este modelo, $r$ es la tasa de crecimiento de las presas y 1 es su "capacidad de carga", podemos observar que si en algún momento $P>1$, $(1-P)<0$ y así, en el siguiente paso, la población disminuye. 

En cambio, si no hubiera presas los depredadores se morirían, ya que no tendrían fuente de alimentos:

$$
\Delta Q = -u Q
$$

Con $u$ positiva y menor a 1, ya que sólo se pueden morir todos los depredadores, pero no más. (En el caso de $u=1$ la población de depredadores desaparecería.)

Para introducir la interacción mutua entre las dos especies empleamos el producto $PQ$. Dado que la interacción con el depredador disminuye a la problación de presas pero beneficia a la de depredadores, obtenemos las siguientes ecuaciones:

$$
\begin{aligned}
\Delta P &= rP\left( 1-P \right)-sPQ\\
\Delta Q &= -uQ+vPQ
\end{aligned}
$$

Donde $s$ y $v$ son constantes positivas que denontan que tanto afecta o beneficia la interacción entre ambas especies a las presas o a los depredadores, respectivamente.

Si escribimos el modelo en términos de la población en lugar de cambios en la población lo que hay que hacer es sumar la población anterior, y obtenemos:


\begin{aligned}
P_{t+1} &= P_t \left( 1+r \left( 1-\frac{P_t}{K} \right)\right)-s P_t Q_t\\
Q_{t+1} &= (1-u)Q_t+v P_t Q_t
\end{aligned}
$$

Con $r$, $s$, $u$ y $v$ constantes positivas, y $u<1$.

Por lo general estas ecuaciones se estudian en el caso continuo, como ecuaciones diferenciales, pero la dinámica en el caso discreto es distinta. Es importante entender este caso también porque las poblaciones no cambian continuamente, así que un modelo discreto tal vez es más adecuado. 

Observamos que el modelo es un mapeo discreto de dos dimensiones, no-lineal y con múltiples parámetros. Definamos la función que define el mapeo:

In [2]:
"""
    CP_Simple(Xt,constantes)

Dado un vector con los valores de la población de presas y depredadores 
Xt=[P,Q], realiza el siguiente paso con el modelo simple.

"""
function CP_Simple(Xt::Vector{Float64},constantes::Vector{Float64})
    
    @assert length(Xt)==2 && length(constantes) == 5
    
    r = constantes[1]
    s = constantes[2]
    u = constantes[3]
    v = constantes[4]
    K = constantes[5]
    
    #Asignación del paso 0
    P0 = Xt[1]
    Q0 = Xt[2]
    #Evaluación del modelo
    P1 = P0*(1+r*(1-P0/K))-s*P0*Q0
    Q1 = (1-u)*Q0+v*P0*Q0
    
    #Resultados
    [P1,Q1]   
end

CP_Simple

In [3]:
"""
    mapeo2D_lista(F,X_0,constantes,t)
Dada una función `F(X_0,constantes)` que define un mapeo de 2 dimensiones
un punto inicial `X_0` y las `constantes` en forma de vector, regresa dos
vectores con los `t` iterados del mapeo para cada dimensión.

"""

function mapeo2D_lista(F::Function , X_0::Vector{Float64},constantes,t::Int64)
    @assert length(X_0)==2 && t > 0
    
    X0 = X_0
    #Definición de los vectores para almacenar resultados.
    Ps = [X0[1]]
    Qs = [X0[2]]
    
    for i in 1:t-1
        X0 = F(X0,constantes)
        push!(Ps,X0[1])
        push!(Qs,X0[2])
    end
    Ps,Qs
end

mapeo_PQ_ts(v_0,r,s,u,v,t) = mapeo_PQ_ts(v_0,r,s,u,v,1.0,t)

mapeo_PQ_ts (generic function with 1 method)

Veamos un ejemplo con parámetros específicos:

In [4]:
r,s,u,v,K = 1.2,.5,.7,1.6,1.
const1 = [r,s,u,v,K]
X0 = [1.10,0.40]

Ps1,Qs1 = mapeo2D_lista(CP_Simple,X0,const1,100)

plot(Ps1,label="Depredadores")
plot(Qs1,label="Presas")
xlabel(L"t")
ylabel("Población")
legend()
title("Ejemplo del Modelo Simple")

LoadError: LoadError: UndefVarError: plot not defined
while loading In[4], in expression starting on line 7

El gráfico muestra las características clásicas de estos modelos, poblaciones que oscilan alrededor de un punto de queilibrio, esto es más notorio si se grafica el diagrama de fase:

In [5]:
plot(Ps1,Qs1)
xlabel(L"P")
ylabel(L"Q")
title("Diagrama de fase del modelo Simple")

LoadError: LoadError: UndefVarError: plot not defined
while loading In[5], in expression starting on line 1

## Puntos fijos

Sin perder generalidad, podemos hacer $K=1$, que de todas formas puede ser absorbida por un cambio de unidades de $P$ de la forma $P'=P/K$ y absorviendo la $K$ en las constantes.

Buscamos $(P,Q)$ tal que 

$$
\begin{aligned}
P &= P \left( 1+r \left( 1-P \right)\right)-s P Q\\
Q &= (1-u)Q+v P Q
\end{aligned}
$$

Los puntos fijos son:


\begin{aligned}
E_0 &= [0.0 ,0.0]\\
E_1 &= [1.0 ,0.0]\\
E_2 &= \left[\frac{u}{v} ,\frac{r}{s}\left(1-\frac{u}{ v} \right) \right]
\end{aligned}

Como el modelo es de poblaciones, los puntos fijos para existir deben permanecer positivos. $E_0$ y $E_1$ siempre existen, para que la condición se cumpla en $E_3$ tenemos:

$$ 
v>u
$$

El jacobiano del mapeo para $E_2$:

$$
J(E_2)=
\begin{pmatrix}
1-\frac{ru}{v} & -\frac{su}{v} \\ 
 \frac{r(-u+v)}{s}& 1
\end{pmatrix}
$$

La ecuación característica del Jacobiano:

$$\lambda^2+\alpha_1\lambda+\alpha_2=0$$

con $\alpha_1=-trJ=-2+\frac{ru}{v}$, $\alpha_2=detJ=1+\frac{ru(-1-u+v)}{v}$:

Entonces:

$$
\lambda_{1,2}=-\frac{\alpha_1}{2}\pm\sqrt{\Delta}=1-\frac{ru}{2v}\pm\sqrt{\Delta}
$$

donde $\Delta=\left(\frac{\alpha_1}{2}\right)^2-\alpha_2=\frac{ru\left(ru+4uv-4v^2\right)}{4v^2}$

Si $\Delta>0$, entonces $r_\Delta>-\frac{4(uv-v^2)}{u}$, de lo contrario los eigenvalores son complejos.

Seguramente cuando el valor de $\Delta$ cambie de imaginario a real ocurrirá un cambio de comportamiento.


Con los 
$$
\begin{aligned}
\lambda_{1,2} = 1 &\Rightarrow  r_{\lambda_{1,2}=1}=0 \\
\lambda_{1,2} =0 &\Rightarrow  r_{\lambda_{1,2}=0}=\frac{v_{\lambda_1=0}}{u+u^2-uv}\\
\lambda_{1,2} =-1 &\Rightarrow  r_{\lambda_{1,2}=-1}=\frac{4v}{u(2+u-v)}
\end{aligned}
$$


Ahora, igualamos 
$$
\begin{aligned}
r_\Delta=r_{\lambda_{1,2}=0} &\Rightarrow  v_{\lambda_{1,2}=0}=\frac{1}{2}(1+2u) \\
r_\Delta=r_{\lambda_{1,2}=1} &\Rightarrow  v_{\lambda_{1,2}=0}=1+u
\end{aligned}
$$

Tenemos que revisar tres intervalos, para distintas $r's$, con enfásis en $r_\Delta$. Como observación, vemos que la constante $s$ no define la dinámica, simplemente se limita a cambiar la posición del punto de equilibro $E_2$.



## Bifurcaciones

Para encontrar las bifuraciones debemos fijar los parámetros y variar solamente uno de los parámetros, en este caso $r$.

In [6]:
#Nuevo método de la función mapeo2D_lista, para quedarse sólo con cierto número de valores finales `tout`
function mapeo2D_lista(F::Function , X_0::Vector{Float64},constantes,tin::Int64,tout::Int64)
    
    @assert length(X_0)==2 &&  tin > 0 && tout > 0
    
    X0 = X_0
    
    for i in 1:tin
        X0 = F(X0,constantes)
    end
    
    mapeo2D_lista(F , X0, constantes,tout)

end

# Otro metodo, que emplea una función para determinar el punto inicial donde empieza el mapeo.
function mapeo2D_lista(F::Function , p::Function,constantes,tin::Int64,tout::Int64)
    
    X0 = p(constantes)
    
        mapeo2D_lista(F, X0,constantes,tin,tout)
end

mapeo2D_lista (generic function with 3 methods)

In [7]:
function diagbifurc2D(F::Function, X0, rrange::Range{Float64}, tin::Int64,tout::Int64)
    
    #Definición de variables
    X = Vector{Float64}(tout)
    Y = Vector{Float64}(tout)
    
    xx = Array{Float64,2}(tout, length(rrange))
    yy = Array{Float64,2}(tout, length(rrange))
    
    for ic in eachindex(rrange)
        r = rrange[ic]
        X,Y = mapeo2D_lista(F, X0,r,tin,tout)
        
        xx[:,ic] = X
        yy[:,ic] = Y
    end
    
    return xx,yy
    
end    

diagbifurc2D (generic function with 1 method)

In [8]:
#Función para generar los valores del punto fijo E_2
function pE2(constantes)
    r = constantes[1]
    s = constantes[2]
    u = constantes[3]
    v = constantes[4]
    [u/v,r*(1-u/v)/s]
end

pE2 (generic function with 1 method)

### Doblamiento de periódo.
Primero, si tenemos $Q=0.0$, entonces el mapeo resulta como:

$$P_{t+1} = P_t \left( 1+r \left( 1-P_t \right)\right)$$

$$P_{t+1} =  r+P_t-rP_t^2 $$

Se devería esperar un comportamiento muy similar al del mapeo cuadrático.


In [9]:
#Debido a una multitud de gráficos similares, esta función ayuda a graficar.
function plot1(F::Function, X_0, range::Range, tin::Int64, tout::Int64, titulo::String)
    
    Ps,Qs=diagbifurc2D(F,X_0, range, tin,tout)
    Rs = ones(size(Ps)[1])*range'
    
    #Cambio de forma de las matrices.
    Ps = reshape(Ps, size(Ps)[1]*size(Ps)[2]);
    Qs = reshape(Qs, size(Qs)[1]*size(Qs)[2]);
    Rs = reshape(Rs, size(Rs)[1]*size(Rs)[2]);
    
    #Plots
    plot(Rs,Ps,"b,",label=(L"P"))
    plot(Rs,Qs,"r,",label=(L"Q"))
    xlabel(L"r")
    ylabel(L"P_{\infty},Q_{\infty}")
    title(titulo)
    legend()
    #Resultados, en caso de usarse
    Rs,Ps,Qs
end

plot1(F::Function, X_0, range::Range, titulo::String)=plot1(F, X_0, range,10000,500, titulo::String)

LoadError: LoadError: UndefVarError: @L_str not defined
while loading In[9], in expression starting on line 2

In [10]:
rrange1 = 1.5:2.0^-10:3.0
s,u,v,K = 3.0,1.25,1.95,1.0
X0 = [0.641,0.000]

F1(Xt,r)=CP_Simple(Xt,[r,s,u,v,K]);

plot1(F1, X0, rrange1, "Diagrama de bifurcación para Q=0.0");

LoadError: LoadError: UndefVarError: plot1 not defined
while loading In[10], in expression starting on line 7

Obtenemos un diagrama de bifurcación de doblamiento de periódo. Sin embargo, no nos interesa el análisis de éste, debido a que lo importante aquí es la dinámica de las dos componentes, no sólo de una. Análicemos para puntos cercanos a $Q=0.0$.

In [11]:
for i in 1:4
    X0 = [0.641,10.0^(-i)]
    Ps4,Qs4=diagbifurc2D(F1,X0, rrange1, 10000,500)
    Rs4 = ones(size(Ps4)[1])*rrange1'

    # Esto cambia las matrices en vectores; ayuda un poco para los dibujos
    Ps4 = reshape(Ps4, size(Ps4)[1]*size(Ps4)[2]);
    Qs4 = reshape(Qs4, size(Qs4)[1]*size(Qs4)[2]);
    Rs4 = reshape(Rs4, size(Rs4)[1]*size(Rs4)[2]);
    
    subplot(220+i)
    plot(Rs4,Ps4,"b,")
    plot(Rs4,Qs4,"r,")
    if i==3 && i==4 
        xlabel(L"r")
    end
    if i==1 && i==3 
        ylabel(L"P_{\infty}")
    end
    title(L"x_0="*"$X0")
    
end

LoadError: LoadError: UndefVarError: @L_str not defined
while loading In[11], in expression starting on line 1

Observemos que alrededor del $r\sim2.8$, ocurre un cambio de comportamiento. Creemos una función para determinar este cambio de regimen.

In [12]:
"""
    ordenar_eliminar_val_rep!(vec::Vector{Float64},ϵ::Float64)
Dado un vector `vec`, ordena y quita los elementos repetidos del vector con una tolerancia `ϵ`
"""
function ordenar_eliminar_val_rep!(vector::Vector{Float64},ϵ::Float64)
    
    N=length(vector)
    
    sort!(vector) #Ordena el vector.
    
    k = 1
    
    for i in 2:N          
        if abs(vector[k]-vector[k+1]) <= ϵ  #Obseva si los valores difieren más allá de epsilón.
            deleteat!(vector,k)             #Elimina el valor repetido.            
        else
            k=k+1
        end     
    end
    
    vector
    
end

#Método con ϵ predefinida.

ordenar_eliminar_val_rep!(vec)=ordenar_eliminar_val_rep!(vec,1e-5)

ordenar_eliminar_val_rep! (generic function with 2 methods)

Ahora, creamos una función que busque un aumento súbito de elementos en dos vectores, en un intervalo.

In [13]:
function mapeo2D_bif(F::Function , X_0, rrange::Range, tin::Int64,tout::Int64)
    #variable para almacenar el resultado
    rresultado=Float64
    
    for i in length(rrange)-1
        #función para r
        r(j)=rrange[j]
        
        #Definición de los dos vectores.
        vec1P,vec1Q=mapeo2D_lista(F, X_0,r(i)  ,tin,tout)
        vec2P,vec1Q=mapeo2D_lista(F, X_0,r(i+1),tin,tout)
        
        #Ordenamiento de los vectores.
        ordenar_eliminar_val_rep!(vec1P)
        ordenar_eliminar_val_rep!(vec2P)
        
        #Condición.        
        if length(vec1P)!=length(vec2P)
            rresultado=(r(i)+r(i+1))/2
            break
        end        
    end
    
    rresultado
end

mapeo2D_bif (generic function with 1 method)

Busquemos la región donde ocurre la bifurcación.

In [14]:
rrange1=2.8:2.0^-20:2.85
mapeo2D_bif(F1,[0.641,0.01], rrange1, 10000,500)

2.8499987602233885

In [15]:
mapeo2D_bif(F1,[0.641,0.001], rrange1, 10000,500)

2.8499987602233885

In [16]:
mapeo2D_bif(F1,[0.641,0.0001], rrange1, 10000,500)

2.8499987602233885

Observamos, que, como era de esperarse, la constante parece invariante en esa región. Grafiquemos el diagrama de fase en ese punto.

In [17]:
#Otra función para graficos recurrentes
function plot2(mapeo::Function, X_0, constantes, t::Int64, titulo::String,tipo::String)
    Ps,Qs = mapeo2D_lista(mapeo,X_0,constantes,t)
    
    #plots
    plot(Ps,Qs,tipo)
    xlabel(L"P")
    ylabel(L"Q")
    title(titulo)
    return Ps,Qs    
end

#Función con el tipo de gráfico estándar
plot2(mapeo::Function, X_0, constantes, t::Int64, titulo::String)=plot2(mapeo, X_0, constantes, t, titulo,"b")

#Función con varias cosas ya definidas.
plot2(mapeo::Function, X_0, constantes, titulo::String)=plot2(mapeo, X_0, constantes,100, titulo)

#Para ignorar los primeros tin
function plot2(mapeo::Function, X_0, constantes, tin::Int64,tout::Int64, titulo::String,tipo::String)
    Ps,Qs = mapeo2D_lista(mapeo,X_0,constantes,tin,tout)
    
    #plots
    plot(Ps,Qs,tipo)
    xlabel(L"P")
    ylabel(L"Q")
    title(titulo)
    return Ps,Qs    
end

LoadError: LoadError: UndefVarError: @L_str not defined
while loading In[17], in expression starting on line 2

In [18]:
r = 2.8499987602233885
const1 = [r,s,u,v,K]
X0 = [0.641,0.01]

plot2(CP_Simple,X0,const1,"Gráfico en la bifurcación");


LoadError: LoadError: UndefVarError: plot2 not defined
while loading In[18], in expression starting on line 5

Se tiene un caso, que parece ser de periódo 3, en el que las depredadores se extinguen, y queda el comportamiento oscilante de las presas, cuyo equilibrio se define por el diagrama de bifurcación.

## Bifurcación del punto de equilibrio. Otro caso.

Si ahora, buscamos cerca de las zona donde los eigenvalores del Jacobiano en el punto de equilibro $E_2$ se vuelven imaginarios. Es decir $r_\Delta<-\frac{4(uv-v^2)}{u}$

In [19]:
s,u,v,K = 3.0,1.25,1.95,1.0
-4*(u*v-v^2)/u

4.367999999999999

In [20]:
rrange1 = 4.2:2.0^-11:5.0

#La función p sirve para definir valores iniciales cercanobs al punto de equilibrio
p4(r)=[0.641,pE2([r,s,u,v])[2]-0.061]

plot1(F1, p4, rrange1, "Valores cercanos a los Eigenvalores complejos");

LoadError: LoadError: UndefVarError: plot1 not defined
while loading In[20], in expression starting on line 6

La dinámica en el punto parece ser normal, las condiciones se van al punto de equilibrio, sin embargo aparece una zona de comportamiento anormal.

In [21]:
#Otro metodo que permite graficar lineas que separan regiones
function plot1(F::Function, X_0, range::Range, A::Vector{Float64}, titulo::String)
    for i in 1:length(A)
        plot([A[i],A[i]],[0.36,0.80],"g")
    end
    plot1(F, X_0, range, titulo)
end

plot1 (generic function with 1 method)

In [22]:
rrange1 = 4.76:2.0^-13:4.9

#La función p sirve para definir valores iniciales cercanobs al punto de equilibrio
p4(r)=[0.641,pE2([r,s,u,v])[2]-0.061]

#Lineas que dividen las regiones
A=[4.782921348,4.785393258,4.810337079,4.814157303,4.863595506,4.877752809]

Rs4,Ps4,Qs4=plot1(F1, p4, rrange1, A, "Zona de bifurcaciones en 2 dimensiones");

LoadError: LoadError: UndefVarError: plot not defined
while loading In[22], in expression starting on line 9

Observamos que tenemos regiones de comportamiento extraño, y ¡Una zona con bifurcaciones en 2 dimensiones!, enseguida graficamos en dos dimensiones el diagrama de fase.

In [23]:
plot3D(Rs4,Ps4,Qs4,"b,")
xlabel(L"r")
ylabel(L"P_\infty")
zlabel(L"Q_\infty")
title("Zona de bifurcaciones en el espacio")

LoadError: LoadError: UndefVarError: plot3D not defined
while loading In[23], in expression starting on line 1

Tenemos tres regiones de interés, marcadas entre líneas verdes:

In [24]:
rrange1 = 4.782921348:2.0^-17:4.785393258
Rs,Ps6,Qs6 = plot1(F1, p4, rrange1, 100000,3000, "Región 1");

LoadError: LoadError: MethodError: no method matching plot1(::#F1, ::#p4, ::FloatRange{Float64}, ::Int64, ::Int64, ::String)
Closest candidates are:
  plot1(::Function, ::Any, ::Range{T}, !Matched::Array{Float64,1}, !Matched::String) at In[21]:3
while loading In[24], in expression starting on line 2

Se observa una región de extrañas bifurcaciones, entre zonas muy ruidosas. Parece ser la superposición de distintos comportamientos.

La región 2 tiene un comportamiento más sencillo de definir.

In [25]:
rrange1 = 4.810337079:2.0^-17:4.814157303
plot1(F1, p4, rrange1, 100000,3000, "Región 2");

LoadError: LoadError: MethodError: no method matching plot1(::#F1, ::#p4, ::FloatRange{Float64}, ::Int64, ::Int64, ::String)
Closest candidates are:
  plot1(::Function, ::Any, ::Range{T}, !Matched::Array{Float64,1}, !Matched::String) at In[21]:3
while loading In[25], in expression starting on line 2

En el siguiente gráfico vemos como hay una región donde el mapeo tiende a 4 zonas, que ha medida que el valor de $r$, se vuelven más cerradas.

In [26]:
r = 4.81+0.001
X0 =  p4(r)
plot2(F1, X0, r, 2000,500, "Curvas de convergencia en la región 2 con r=$r","b,");

LoadError: LoadError: UndefVarError: plot2 not defined
while loading In[26], in expression starting on line 3

Aquí ya las zonas están bastante cerradas.

In [27]:
r = 4.81+0.004
X0 =  p4(r)
plot2(F1, X0, r, 2000,500, "Curvas de convergencia en la región 2 con r=$r","b,");

LoadError: LoadError: UndefVarError: plot2 not defined
while loading In[27], in expression starting on line 3

In [28]:
plot2(mapeo::Function, X_0, constantes, t::Int64, titulo::String)

LoadError: LoadError: UndefVarError: plot2 not defined
while loading In[28], in expression starting on line 1

In [29]:
rrange1 = 4.863595506:2.0^-17:4.877752809
plot1(F1, p4, rrange1, 100000,3000, "Región 3");

LoadError: LoadError: MethodError: no method matching plot1(::#F1, ::#p4, ::FloatRange{Float64}, ::Int64, ::Int64, ::String)
Closest candidates are:
  plot1(::Function, ::Any, ::Range{T}, !Matched::Array{Float64,1}, !Matched::String) at In[21]:3
while loading In[29], in expression starting on line 2

En la región 3 tenemos un comportamiento muy similar al que ocurre en las zonas de doblamiento de periodo de una dimensión, pero en lugar de tener la bifurcación en un plano, la tenemos en el espacio.

In [30]:
subplot(121)
r = 4.866
X0 =  p4(r)
plot2(F1, X0, r, 10000, "Périodo 1, r=$r");

subplot(122)
r = 4.872
X0 =  p4(r)
plot2(F1, X0, r, 10000, "Périodo 2, r=$r");

LoadError: LoadError: UndefVarError: subplot not defined
while loading In[30], in expression starting on line 1

In [31]:
subplot(121)
r = 4.8755
X0 =  p4(r)
plot2(F1, X0, r, 10000, "Périodo 4, r=$r");

subplot(122)
r = 4.8765
X0 =  p4(r)
plot2(F1, X0, r, 10000, "Périodo 8, r=$r");

LoadError: LoadError: UndefVarError: subplot not defined
while loading In[31], in expression starting on line 1

In [32]:
r = 4.814157303
X0 =  p4(r)
plot2(F1, X0, r, 10000, "Périodo grande, r=$r","b,");

LoadError: LoadError: UndefVarError: plot2 not defined
while loading In[32], in expression starting on line 3

Observamos que las bifurcaciones aumentan alrededor de 4 puntos de equilibrio.

## Valores cercanos a $v_{\lambda_{1,2}=0}=1+u$ ##

Iniciamos con las siguientes constantes, con  $v=1+u$

In [33]:
s,u,K = 3.0,0.25,1.0
v=1+u

const2 = [r,s,u,v,K]
p2(r)=pE2([r,s,u,v])-[0.0,0.06]
F2(Xt,r)=CP_Simple(Xt,[r,s,u,v,K]);

In [34]:
s,u,K = 3.0,0.25,1.0
v=1+u

const2 = [r,s,u,v,K]
rrange2 = 0.0:2.0^-8:2.0
plot1(F2, p2, rrange2, 20000,500, "Gráfico con v=1+u");

LoadError: LoadError: MethodError: no method matching plot1(::#F2, ::#p2, ::FloatRange{Float64}, ::Int64, ::Int64, ::String)
Closest candidates are:
  plot1(::Function, ::Any, ::Range{T}, !Matched::Array{Float64,1}, !Matched::String) at In[21]:3
while loading In[34], in expression starting on line 6

Se observa un comportamiento de trompeta: es decir, los valores tienden a una región ancha en lugar de a un punto. También vemos que mientras aumenta la $r$ la región donde converge el mapeo se va reduciendo. Grafiquemos un diagrama de fase.

In [35]:
r = 0.3
X0 =  p2(r)
plot2(F2, X0, r, 10000, "Diagrama de fase con r=$r y v=1+u, para los primeros n=10000","b,");

LoadError: LoadError: UndefVarError: plot2 not defined
while loading In[35], in expression starting on line 3

Si se aumenta el número de iterados, entonces se puede ver a donde converge el mapeo, una región cerrada del espacio fase.

In [36]:
r = 0.3
X0 =  p2(r)
plot2(F2, X0, r, 20000,500, "Diagrama de fase con r=$r y v=1+u, para los primeros n=10000","b,");

LoadError: LoadError: UndefVarError: plot2 not defined
while loading In[36], in expression starting on line 3

Obtengamos sólo el contorno de las figuras cuando $P_\infty$ y $Q_\infty$. Se hace esto, simplemente obteniendo los máximos.

In [37]:
function mapeo2D_max(F::Function , E2::Function, X_0, rrange::Range, tin::Int64,tout::Int64)
    #variable para almacenar el resultado
    Pres=Float64[]
    Qres=Float64[]
    Rres=Float64[]
    
    for r in rrange        
        #Definición de los dos vectores.
        vecP,vecQ=mapeo2D_lista(F, X_0, r, tin,tout)
        
        #maximos
        maxP=maximum(vecP)-E2(r)[1]
        maxQ=maximum(vecQ)-E2(r)[2]
        
        
        if isnan(maxP) == false && isnan(maxQ) == false
            push!(Rres,r)
            push!(Pres,maxP)
            push!(Qres,maxQ)
        end
    end
    
    Rres, Pres,Qres
end

mapeo2D_max (generic function with 1 method)

También, debemos restar la posición del punto de equilibrio, para poder ver que tanto osc

In [38]:
rrange2 = 0.0:2.0^-8:10.0
pp2(r)=pE2([r,s,u,v])
Rs3,Ps3,Qs3 = mapeo2D_max(F2 ,pp2, p2, rrange2, 20000,500);


Utilizamos un paquete para ajustar la curva.

In [39]:
#Ajuste propuesto, r es la variable, p es un arreglo de constantes.
model(r, p) = p[1]*(r).^(-p[2]).+p[3]
#arreglo con el peso de los valores, en este caso todos son igualmente pesados.
w=ones(length(Rs3))
#Ajuste del modelo.
fitP = curve_fit(model, Rs3, Ps3,w)
fitQ = curve_fit(model, Rs3, Qs3,w);

LoadError: LoadError: UndefVarError: curve_fit not defined
while loading In[39], in expression starting on line 6

El modelo electo es $ P,Q = a(r+b)^{-c}+d$

In [40]:
fitP

LoadError: LoadError: UndefVarError: fitP not defined
while loading In[40], in expression starting on line 1

In [41]:
P1=[0.0132109,0.523979,-0.00187789];

La curva para las presas está definida como $P_{contorno,\infty}\propto \frac{1}{\sqrt{r}}$

In [42]:
fitQ

LoadError: LoadError: UndefVarError: fitQ not defined
while loading In[42], in expression starting on line 1

In [43]:
Q1=[0.0330781,0.0423991,-0.0247437];

In [44]:
subplot(211)
PAs3=[model(r,P1) for r in Rs3]
plot(Rs3,Ps3,"r.",label=("Valores max"))
plot(Rs3,PAs3,label=("Ajuste"))
xlabel(L"r")
ylabel(L"P_{\infty}")
title("titulo")
legend()

subplot(212)

QAs3=[model(r,Q1) for r in Rs3]
plot(Rs3,Qs3,"r.",label=("Valores max"))
plot(Rs3,QAs3,label=("Ajuste"))
xlabel(L"r")
ylabel(L"Q_{\infty}")
title("titulo")
legend()

LoadError: LoadError: UndefVarError: subplot not defined
while loading In[44], in expression starting on line 1

Naturalmente, si el valor de $v$ es ligeramente menor, se pierde este comportamiento, teniendo la convergencia de la bifurcación en un punto.

In [45]:
s,u,K = 3.0,0.25,1.0
v=1+u-0.01

const2 = [r,s,u,v,K]
rrange2 = 0.0:2.0^-8:2.0
plot1(F2, p2, rrange2, 20000,500, "Gráfico con v=1+u-0.01");

LoadError: LoadError: MethodError: no method matching plot1(::#F2, ::#p2, ::FloatRange{Float64}, ::Int64, ::Int64, ::String)
Closest candidates are:
  plot1(::Function, ::Any, ::Range{T}, !Matched::Array{Float64,1}, !Matched::String) at In[21]:3
while loading In[45], in expression starting on line 6

## Valores cercanos a $v=\frac{1}{2}(1+2u)$

In [46]:
s,u,K = 3.0,0.25,1.0
v=0.5(1+2u)
p3(r)=pE2([r,s,u,v])-[0.0,0.06]
F3(Xt,r)=CP_Simple(Xt,[r,s,u,v,K]);
rrange3 = 0.0:2.0^-8:10.0
plot1(F3, p3, rrange3, 20000,500, "Gráfico con v=0.5*(1+2u)");

LoadError: LoadError: MethodError: no method matching plot1(::#F3, ::#p3, ::FloatRange{Float64}, ::Int64, ::Int64, ::String)
Closest candidates are:
  plot1(::Function, ::Any, ::Range{T}, !Matched::Array{Float64,1}, !Matched::String) at In[21]:3
while loading In[46], in expression starting on line 6

In [47]:
rrange3 = 7.9:2.0^-10:9.0
R3,P3,Q3=plot1(F3, p3, rrange3, 20000,500, "Gráfico con v=0.5*(1+2u)");

LoadError: LoadError: MethodError: no method matching plot1(::#F3, ::#p3, ::FloatRange{Float64}, ::Int64, ::Int64, ::String)
Closest candidates are:
  plot1(::Function, ::Any, ::Range{T}, !Matched::Array{Float64,1}, !Matched::String) at In[21]:3
while loading In[47], in expression starting on line 2

In [48]:
plot3D(R3,P3,Q3,"b,")
xlabel(L"r")
ylabel(L"P_\infty")
zlabel(L"Q_\infty")
title("Zona de bifurcaciones en el espacio")

LoadError: LoadError: UndefVarError: plot3D not defined
while loading In[48], in expression starting on line 1

Se tiene un nuevo gráfico, con bifurcaciones de doblamiento de periódo torcidas.

In [49]:
r = 8.4
X0 =  p3(r)
plot2(F3, X0, r, 20000,500, "Diagrama de fase con r=$r y v=0.5(1+2u), para los primeros n=10000","b");

LoadError: LoadError: UndefVarError: plot2 not defined
while loading In[49], in expression starting on line 3

Aunque el comportamiento es el mismo que el que ya se describió, en el otro caso de bifurcaciones de doblamiento de periodo.

In [50]:
#Pkg.add("PyPlot");
#Pkg.add("Combinatorics");
#Pkg.add("StatsBase");
#Pkg.add("DataFrames")
#Pkg.add("Gadfly")
#Pkg.add("Plots");
#Pkg.update();

# Todas las funciones grandes dentro de este programa tienen buenos docs. No los dejes solos...
using PyPlot, Gadfly, StatsBase, Combinatorics,Plots;

LoadError: LoadError: ArgumentError: Module Gadfly not found in current path.
Run `Pkg.add("Gadfly")` to install the Gadfly package.
while loading In[50], in expression starting on line 10

### Iniciando el campo de juego
Simulación de un mundo discreto de depredador - presa discreto

In [51]:
#0.1 Iniciar constantes y arreglos

const NGridSize = 12 :: Int        #Tamaño de Matriz
const TotalTurns = 100 :: Int #Turnos de tiempo global

const α = .5                       #Coeficiente de clon de presa

const β = .5                       #Coeficiente de clon de depredador
const ϵ_inicial = -3 :: Int               #Energia inicial para cada clon de depredador
const Δϵ = 1 :: Int                #Perdida de energia por paso de tiempo

const γ = .1                       #Porcentaje a hacer update en cada turno
const ρ_1 = .333                   #Densidad de presas
const ρ_2 = .333                   #Densidad de depredadores

#0.2 Construir matriz
const items = [-3,-2,-1,0,1]
const weights = [ρ_1 /3, ρ_1 /3, ρ_1 /3, 1-ρ_1-ρ_2, ρ_2]

# Chulada 1 con paquete StatsBase
Grid = reshape(sample(items,WeightVec(weights),NGridSize^2, replace=true)  ,NGridSize,NGridSize) ::Array{Int,2}

LoadError: LoadError: UndefVarError: sample not defined
while loading In[51], in expression starting on line 21

In [52]:
### Reglas de comportamiento

A considerar:

Si es presa
1. Si se puede mover
2. Si se clona

Si es depredador
1. Si puede mover
2. Si come
3. Si se clona

Consideraciones "amables":
- Fronteras toroidales
- Vecinos a distancia 1
- ¿Si puede comer el lobo, come?
- ¿Quién juega primero?

LoadError: LoadError: syntax: extra token "considerar" after end of expression
while loading In[52], in expression starting on line 3

In [53]:
"""
check_vecinos_vacios(Row::Int,Col::Int)

Esta función revisa si en un vecindario de las 8 casillas adyacentes a una entrada en la matriz de Grid
existe al menos un solo vecino vacío. El código toma en cuenta los casos donde la frontera es toroidal.
Ejemplo:
```julia
julia> a=eye[2,2]
julia> check_vecinos_vacios(1,1)
1
julia> check_vecinos_vacios(1,2)
0
"""
function check_vecinos_vacios(Row::Int,Col::Int,Grid::Array{Int,2},NGridSize::Int)
    YSouth = mod(Row,NGridSize)+1
    XWest = mod(Col,  NGridSize) +1
    XEast = mod(Col-2,  NGridSize) +1
    YNorth = mod(Row-2, NGridSize) +1
    
    if Grid[YNorth, Col]== 0 ||Grid[Row, XEast]==0 ||
    Grid[Row, XWest]==0 || Grid[YSouth, Col]==0 ||
    Grid[YNorth,XEast]==0 || Grid[YNorth,XWest]==0 ||
    Grid[YSouth,XEast]==0 || Grid[YSouth,XWest]== 0
        return 1
    
    else return 0
    end
end

"""
check_comer_depredador(Row::Int,Col::Int,Grid::Array{Int,2})

Esta función revisa si en un vecindario de las 8 casillas aledañas a la entrada (Row,Col) existe al menos
una presa, y regresa un 1 en dado caso, 0 c.o.c. El código toma en cuenta el caso donde la frontera es toroidal.
julia> a=eye[2,2]
julia> check_comer_depredador(1,1)
1
julia> b = zeros[2,2]
julia> check_comer_depredador[1,1]
0
"""
function check_comer_depredador(Row::Int,Col::Int,Grid::Array{Int,2},NGridSize::Int)
    
    if Grid[Row,Col] == 0 || Grid[Row,Col] == 1
        return 0
    end
    
    
    YSouth = mod(Row,NGridSize)+1
    XWest = mod(Col,  NGridSize) +1
    XEast = mod(Col-2,  NGridSize) +1
    YNorth = mod(Row-2, NGridSize) +1
    
    if Grid[YNorth, Col] > 0 || Grid[Row, XEast] > 0 || Grid[Row, XWest] > 0 ||  Grid[YSouth,Col] > 0 || Grid[YNorth,XEast] > 0 || Grid[YNorth,XWest] > 0 || Grid[YSouth,XEast] > 0 ||  Grid[YSouth,XWest] > 0
        return 1
    else return 0
    end
end

"""
check_mover_depredador(Row::Int,Col::Int,Grid::Array{Int,2})

Esta función revisa si en un vecindario de las 8 casillas aledañas a la entrada (Row,Col) existe al menos
un movimiento posible para el depredador, y regresa un 1 en dado caso, 0 c.o.c. El código toma en cuenta el caso donde la frontera es toroidal.
julia> a=eye[2,2]
julia> check_mover_depredador(1,1)
1
julia> b = zeros[2,2]
julia> check_comer_depredador[1,1]
0
"""
function check_mover_depredador(Row::Int,Col::Int,Grid::Array{Int,2},NGridSize::Int)
    YSouth = mod(Row,NGridSize)+1
    XWest = mod(Col,  NGridSize) +1
    XEast = mod(Col-2,  NGridSize) +1
    YNorth = mod(Row-2, NGridSize) +1
    
    if Grid[Row,Col] != 0 || Grid[YNorth, Col] >= 0 || Grid[Row, XEast] >= 0 || Grid[Row, XWest] >= 0 ||  Grid[YSouth,Col] >= 0 || Grid[YNorth,XEast] >= 0 || Grid[YNorth,XWest] >= 0 || Grid[YSouth,XEast] >= 0 || Grid[YSouth,XWest] >= 0
        return 1
    else return 0
        
    end
end

check_mover_depredador


### Consideraciones "amables":
- Fronteras toroidales 
    - yup
- Vecinos a distancia 1 
    - yup
- ¿Si puede comer el lobo, come?
    - yup
- ¿Quién juega primero?
    - presa
- ¿Energía de clon de depredador es independiente a si come?
    - yup
- ¿Tope de energía para depredador?
    - yup
- ¿Cuál es la distribución inicial de energía para los lobos?
    - Uniforme, desde la inicial

In [54]:
#Movimiento de presas

"""
moviendo_presa(Row::Int,Col::Int,Grid::Array{Int,2},NGridSize::Int,α::Float64)

Esta función establece el comportamiento de la presa.Si una
presa en (Row,Col) puede moverse a una casilla aledaña, lo haráy si
se clona con probabilidad α.

"""

function moviendo_presa(Row::Int,Col::Int,Grid::Array{Int,2},NGridSize::Int,α::Float64)
    #Sanity check
    if Grid[Row,Col] <= 0
        return 0 
    end
    
    if check_vecinos_vacios(Row,Col,Grid,NGridSize) == 1
        
        #Coeficiente de clonado - a veces deja una Dolly en su lugar...
        rand() < α ? clon = 1 : clon = 0
        
        #Chulada 2 - Fronteras toroidales en cuadrícula. 
        #Considerar usar la función mod1()
        #Con estas defs podemos revisar valores de vecinos.
        YSouth = mod(Row,NGridSize)+1
        XWest = mod(Col,  NGridSize) +1
        XEast = mod(Col-2,  NGridSize) +1
        YNorth = mod(Row-2, NGridSize) +1
        
        #Gran ciclo para intentar moverse a una casilla no ocupada.
        for i in 1:50
        
        #Oh...
        behave = rand(1:1:8)
            
        #Checando que nuestros vecinos sean movidas permitidas
        if behave == 1 && Grid[YNorth,XWest] == 0
            Grid[YNorth,XWest] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 2 && Grid[YNorth,Col] == 0
            Grid[YNorth,Col] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 3 && Grid[YNorth,XEast] == 0
            Grid[YNorth,XEast] = 1 
            Grid[Row,Col] = clon
                break
        elseif behave == 4 && Grid[Row,XWest] == 0
            Grid[Row,XWest] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 5 && Grid[Row,XEast] == 0
            Grid[Row,XEast] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 6 && Grid[YSouth,XWest] == 0
            Grid[YSouth,XWest] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 7 && Grid[YSouth,Col] == 0
            Grid[YSouth,Col] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 8 && Grid[YSouth,XEast] == 0
            Grid[YSouth,XEast] = clon
            Grid[Row,Col] = clon
                break
        
            end
        end
    end
end

            
"""
moviendo_depredador(Row::Int,Col::Int,Grid::Array{Int,2},NGridSize::Int,β::Float64)

Establece reglas de comportamiento para el depredador en (Row,Col). Si puede comer a una presa
aledaña, preferirá comer y moverse a sólo moverse. Se puede reproducir con
probabilidad β.
"""

#Movimiento de depredadores
function moviendo_depredador(Row::Int,Col::Int,Grid::Array{Int,2},NGridSize::Int,β::Float64)
    
    #Sanity check
    if Grid[Row,Col] >= 0
        return 0
    end
    
    puede_moverse = 0
    puede_comer = 0
    
    #Verificamos que se puedan mover
    puede_moverse = check_mover_depredador(Row,Col,Grid,NGridSize)
    puede_comer   = check_comer_depredador(Row,Col,Grid,NGridSize)
    
    prioridad = puede_moverse + puede_comer
        
    #Asignamos condicional a la ϵ_depredador si puede comer. Esta es la ϵ que se queda el
    # "original", per se. 
    if puede_comer == 1
        Grid[Row,Col] = ϵ_inicial
        else 
        Grid[Row,Col] = Grid[Row,Col] + Δϵ
    end
    
    #Starvation check
    if Grid[Row,Col] >= 0 
        return 0
    end
    
    ϵ_depredador = Grid[Row,Col]
    clon = 0::Int
    
    if check_mover_depredador(Row,Col,Grid,NGridSize) == 1
        
        #Coeficiente de clonado
        rand() < β ? clon = ϵ_inicial : clon = 0
        
        #Otra vez definimos a las casillas vecinas en fronteras toroidales
        YSouth = mod(Row,NGridSize)+1
        XEast = mod(Col,  NGridSize) +1
        XWest = mod(Col-2,  NGridSize) +1
        YNorth = mod(Row-2, NGridSize) +1
                
        #Con puro colmillo no ponemos un while por si las dudas...
        for i in 1:50
        
        behave = rand(1:1:8)
            
            # Chulada 3 - La segunda condicional impera que si puede comer y moverse, debe comer cuando se mueve.
            # ¿Ves cómo?
            if behave == 1 && Grid[YNorth,XWest] == puede_comer
                    Grid[YNorth,XWest] = ϵ_depredador
                    Grid[Row,Col] = clon    
                    break
                elseif behave == 2 && Grid[YNorth,Col] == puede_comer
                    Grid[YNorth,Col] = ϵ_depredador
                    Grid[Row,Col] = clon
                    break
                elseif behave == 3 && Grid[YNorth,XEast] == puede_comer
                    Grid[YNorth,XEast] = ϵ_depredador 
                    Grid[Row,Col] = clon
                    break
                elseif behave == 4 && Grid[Row,XWest] == puede_comer
                    Grid[Row,XWest] = ϵ_depredador
                    Grid[Row,Col] = clon
                    break
                elseif behave == 5 && Grid[Row,XEast] == puede_comer
                    Grid[Row,XEast] = ϵ_depredador
                    Grid[Row,Col] = clon
                    break
                elseif behave == 6 && Grid[YSouth,XWest] == puede_comer
                    Grid[YSouth,XWest] = ϵ_depredador
                    Grid[Row,Col] = clon
                    break
                elseif behave == 7 && Grid[YSouth,Col] == puede_comer
                    Grid[YSouth,Col] = ϵ_depredador
                    Grid[Row,Col] = clon
                    break
                elseif behave == 8 && Grid[YSouth,XEast] == puede_comer
                    Grid[YSouth,XEast] = ϵ_depredador 
                    Grid[Row,Col] = clon
                    break
            
            end
        end
    end
end

 

moviendo_depredador

In [55]:
### ¿Pifa?

In [56]:
NGridSize = 5;Grid = eye(Int,5,5); Grid[2,2] = -3; Grid

5×5 Array{Int64,2}:
 1   0  0  0  0
 0  -3  0  0  0
 0   0  1  0  0
 0   0  0  1  0
 0   0  0  0  1

In [57]:
moviendo_presa(4,4,Grid,NGridSize,.9)
Grid

5×5 Array{Int64,2}:
 1   0  0  0  0
 0  -3  0  0  0
 0   0  1  1  0
 0   0  0  1  0
 0   0  0  0  1

In [58]:
ϵ_inicial = -3
Δϵ = 1

moviendo_depredador(2,2,Grid,NGridSize, 0.)
Grid

5×5 Array{Int64,2}:
 -3  0  0  0  0
  0  0  0  0  0
  0  0  1  1  0
  0  0  0  1  0
  0  0  0  0  1

### The Big Loop!

In [59]:
#Super Gran Loop

"""
BigLoop(NGridSize::Int64, α::Float64, β::Float64, ϵ_inicial::Int64, Δϵ::Int64, γ::Float64, ρ_1::Float64, ρ_2::Float64, TurnTimer::Int64)

Esta función toma una familia de parámetros para simular un microcosmos de depredadores y presas en una matriz con
fronteras toroidales. Regresa los arreglos Presas, Depredadores, Turnos, Grid para poder graficarlos.
Primero define sus arreglos y coordenadas, empieza contadores, y luego en un gran loop de iteración evoluciona
los agentes de cambio con mover_depredador() y mover_presa(), para despues contar a los sobrevivientes y push! a arreglos.

"""
function BigLoop(
    
    NGridSize::Int,
    α::Float64,
    β::Float64,
    ϵ_inicial :: Int,
    Δϵ :: Int,
    γ :: Float64,
    ρ_1 :: Float64,
    ρ_2 :: Float64, 
    TurnTimer::Int,)
    #######################
    
    #Iniciar Matriz
    Espacio_Coordenadas = [(x,y) for x=1:NGridSize, y=1:NGridSize]
    items_0 = [-3,-2,-1,0,1]
    weights_0 = [ρ_1 /3, ρ_1 /3, ρ_1 /3, 1-ρ_1-ρ_2, ρ_2]
    Grid = reshape(sample(items_0,WeightVec(weights_0),NGridSize^2, replace=true),NGridSize,NGridSize) ::Array{Int,2}
    
    # Iniciar contadores y randomseed (para que sea replicable los resultados)
    srand(1234)
    iter=enumerate(Grid)
    Presas = [0]
    Depredadores = [0]
    Turnos = [0]
    
    #Chulada 4 del paquete de Combinatorics
    presas_vivas = count(iter->(iter>0), Grid)
    depredadores_vivos = count(iter->(iter<0), Grid)
    
    #Listos para pushear y graficar...
    push!(Presas,presas_vivas)
    push!(Depredadores, depredadores_vivos)
    
    #Variable auxiliar para recorrer sampleo
    tope = convert(Int,round(γ*NGridSize^2))   
    turn_counter = 0
    #The Big Loop-de-Loop!
    
    
    @inbounds for time_step in 1:TurnTimer


        #Ojo, esto se alenta en matrices esparsas...por...?
        aux = sample(Espacio_Coordenadas,tope;replace=false) :: Array{Tuple{Int64,Int64},1}
       
        #Chulada 5 con reinterpret y vector - Como convertir tuplas en arreglos
        sampleo = vec(reinterpret(Int, aux, (1,tope*2)))
        turn_counter += 1
        
        @inbounds for i in 1:tope*2-1

            Row,Col = sampleo[i], sampleo[i+1]

            # Who moves first? :D
            # Si las entradas no son una casilla vacía, entonces la registramos en todos nuestros
            # Arreglos, si no, simplemente se lo salta
            if Grid[Row,Col] != 0 
                #Movemos a las presas o depredadores
                moviendo_presa(Row,Col,Grid,NGridSize,α)
                moviendo_depredador(Row,Col,Grid,NGridSize,β)

            end
        end
                #Contamos a los vivos                
                presas_vivas = count(iter->(iter>0), Grid)
                depredadores_vivos = count(iter->(iter<0), Grid)
                #Guardamos cuantos vivos hay
                push!(Presas,presas_vivas)
                push!(Depredadores, depredadores_vivos)
                #Guardamos el turno
                push!(Turnos, turn_counter)
    end
    
    print("\n Simulation complete! \n!")
    return Presas,Depredadores,Turnos, Grid
    
end
    
    
    

BigLoop

### Simulando...



In [60]:
const NGridSize = 12 :: Int        #Tamaño de Matriz
const TurnTimer = 100 :: Int       #Turnos de tiempo global

const α = .7                       #Coeficiente de clon de presa

const β = .3                       #Coeficiente de clon de depredador
const ϵ_inicial = -3::Int              #Energia inicial para cada clon de depredador
const Δϵ = 1 :: Int                #Perdida de energia por paso de tiempo

const γ = .2                       #Porcentaje a hacer update en cada turno
const ρ_1 = .333                   #Densidad de presas
const ρ_2 = .333                   #Densidad de depredadores

#######################################


Presas, Depredadores, Turnos,Result_Grid = BigLoop(NGridSize,α,β,ϵ_inicial,Δϵ,γ,ρ_1,ρ_2,TurnTimer)
presas = Presas[end]; depredadores = Depredadores[end]; turno = Turnos[end]

im = imshow(Result_Grid)
title("Población de presas y depredadores en un mundo toroidal")
################

LoadError: LoadError: UndefVarError: sample not defined
while loading In[60], in expression starting on line 17

In [61]:
p = Plots.plot([Presas,Depredadores],zeros(0),leg=true)
title!("Población de Presas y Depredadores vs Turnos")
yaxis!("N de presas y depredadores",0:TurnTimer/10:TurnTimer)
xaxis!("N de turnos", 0:1:TurnTimer)

anim = Animation()
@gif for x in 1:1:TurnTimer
    push!(p,x,Int[Presas[x],Depredadores[x]])
    frame(anim)
end

LoadError: LoadError: UndefVarError: Plots not defined
while loading In[61], in expression starting on line 1

In [62]:
const NGridSize = 12 :: Int        #Tamaño de Matriz
const TurnTimer = 100 :: Int       #Turnos de tiempo global

const α = .8                       #Coeficiente de clon de presa

const β = .05                       #Coeficiente de clon de depredador
const ϵ_inicial = -3::Int              #Energia inicial para cada clon de depredador
const Δϵ = 1 :: Int                #Perdida de energia por paso de tiempo

const γ = .2                       #Porcentaje a hacer update en cada turno
const ρ_1 = .333                   #Densidad de presas
const ρ_2 = .333                   #Densidad de depredadores
#######################################


Presas, Depredadores, Turnos,Result_Grid = BigLoop(NGridSize,α,β,ϵ_inicial,Δϵ,γ,ρ_1,ρ_2,TurnTimer)
presas = Presas[end]; depredadores = Depredadores[end]; turno = Turnos[end]

im = imshow(Result_Grid)
title("Población de presas y depredadores en un mundo toroidal")
################

LoadError: LoadError: UndefVarError: sample not defined
while loading In[62], in expression starting on line 16

In [63]:
p = Plots.plot([Presas,Depredadores],zeros(0),leg=true)
title!("Población de Presas y Depredadores vs Turnos")
yaxis!("N de presas y depredadores",0:TurnTimer/10:TurnTimer)
xaxis!("N de turnos", 0:1:TurnTimer)

anim = Animation()
@gif for x in 1:1:TurnTimer
    push!(p,x,Int[Presas[x],Depredadores[x]])
    frame(anim)
end

LoadError: LoadError: UndefVarError: Plots not defined
while loading In[63], in expression starting on line 1

In [64]:
const NGridSize = 20 :: Int        #Tamaño de Matriz
const TurnTimer = 100 :: Int       #Turnos de tiempo global

const α = .9                       #Coeficiente de clon de presa

const β = .0123                      #Coeficiente de clon de depredador
const ϵ_inicial = -2::Int              #Energia inicial para cada clon de depredador
const Δϵ = 1 :: Int                #Perdida de energia por paso de tiempo

const γ = .2                       #Porcentaje a hacer update en cada turno
const ρ_1 = .333                   #Densidad de presas
const ρ_2 = .1                     #Densidad de depredadores
#######################################


Presas, Depredadores, Turnos,Result_Grid = BigLoop(NGridSize,α,β,ϵ_inicial,Δϵ,γ,ρ_1,ρ_2,TurnTimer)
presas = Presas[end]; depredadores = Depredadores[end]; turno = Turnos[end]

im = imshow(Result_Grid)
title("Población de presas y depredadores en un mundo toroidal")
################

LoadError: LoadError: UndefVarError: sample not defined
while loading In[64], in expression starting on line 16

In [65]:
p = Plots.plot([Presas,Depredadores],zeros(0),leg=true)
title!("Población de Presas y Depredadores vs Turnos")
yaxis!("N de presas y depredadores",0:TurnTimer/10:TurnTimer)
xaxis!("N de turnos", 0:1:TurnTimer)

anim = Animation()
@gif for x in 1:1:TurnTimer
    push!(p,x,Int[Presas[x],Depredadores[x]])
    frame(anim)
end

LoadError: LoadError: UndefVarError: Plots not defined
while loading In[65], in expression starting on line 1

In [66]:
const NGridSize = 50 :: Int        #Tamaño de Matriz
const TurnTimer = 300 :: Int       #Turnos de tiempo global

const α = .6                       #Coeficiente de clon de presa

const β = .02                       #Coeficiente de clon de depredador
const ϵ_inicial = -5::Int              #Energia inicial para cada clon de depredador
const Δϵ = 1 :: Int                #Perdida de energia por paso de tiempo

const γ = .2                       #Porcentaje a hacer update en cada turno
const ρ_1 = .333                   #Densidad de presas
const ρ_2 = .2                   #Densidad de depredadores
#######################################


Presas, Depredadores, Turnos,Result_Grid = BigLoop(NGridSize,α,β,ϵ_inicial,Δϵ,γ,ρ_1,ρ_2,TurnTimer)
presas = Presas[end]; depredadores = Depredadores[end]; turno = Turnos[end]

im = imshow(Result_Grid)
title("Población de presas y depredadores en un mundo toroidal")
################

LoadError: LoadError: UndefVarError: sample not defined
while loading In[66], in expression starting on line 16

In [67]:
p = Plots.plot([Presas,Depredadores],zeros(0),leg=true)
title!("Población de Presas y Depredadores vs Turnos")
yaxis!("N de presas y depredadores",0:TurnTimer/10:TurnTimer)
xaxis!("N de turnos", 0:1:TurnTimer)

anim = Animation()
@gif for x in 1:1:TurnTimer
    push!(p,x,Int[Presas[x],Depredadores[x]])
    frame(anim)
end

LoadError: LoadError: UndefVarError: Plots not defined
while loading In[67], in expression starting on line 1

In [68]:
const NGridSize = 100 :: Int        #Tamaño de Matriz
const TurnTimer = 100 :: Int       #Turnos de tiempo global

const α = .55                       #Coeficiente de clon de presa

const β = .025                       #Coeficiente de clon de depredador
const ϵ_inicial = -3::Int              #Energia inicial para cada clon de depredador
const Δϵ = 1 :: Int                #Perdida de energia por paso de tiempo

const γ = .2                       #Porcentaje a hacer update en cada turno
const ρ_1 = .8                     #Densidad de presas
const ρ_2 = .1                   #Densidad de depredadores
#######################################


Presas, Depredadores, Turnos,Result_Grid = BigLoop(NGridSize,α,β,ϵ_inicial,Δϵ,γ,ρ_1,ρ_2,TurnTimer)
presas = Presas[end]; depredadores = Depredadores[end]; turno = Turnos[end]

im = imshow(Result_Grid)
title("Población de presas y depredadores en un mundo toroidal")
################

LoadError: LoadError: UndefVarError: sample not defined
while loading In[68], in expression starting on line 16

In [69]:
p = Plots.plot([Presas,Depredadores],zeros(0),leg=true)
title!("Población de Presas y Depredadores vs Turnos")
yaxis!("N de presas y depredadores",0:TurnTimer/10:TurnTimer)
xaxis!("N de turnos", 0:1:TurnTimer)

anim = Animation()
@gif for x in 1:1:TurnTimer
    push!(p,x,Int[Presas[x],Depredadores[x]])
    frame(anim)
end

LoadError: LoadError: UndefVarError: Plots not defined
while loading In[69], in expression starting on line 1

### Bibliografía
 Wilenskz, Reisman,2006. "Thinking Like a Wolf, a Sheep, or a Firefly:Learning Biology Through Constructing and Testing Computational Theories--an Embodied Modeling Approach", http://ccl.northwestern.edu/papers/wolfsheep.pdf